I'll explain the model code in detail. Let me first read the main training file to give you a comprehensive explanation. 

Read [](file:///Users/aksha/Desktop/medical-image-classifier-main/src/model_training.py)

# 📚 Detailed Explanation of the Medical Image Classifier Model

Let me break down this code into digestible sections:

## 1️⃣ **Overall Architecture Overview**

This is a **Convolutional Neural Network (CNN)** trained to classify chest X-ray images into two categories:
- **Normal** (healthy lungs) - labeled as `0`
- **Pneumonia** (infected lungs) - labeled as `1`

The model uses **TensorFlow/Keras**, which makes it easy to build and train deep learning models.

---

## 2️⃣ **Class: MedicalImageClassifier**



In [ ]:
class MedicalImageClassifier:
    def __init__(self, img_height=224, img_width=224):



**Purpose**: Wraps the entire training pipeline into organized methods.

**Parameters**:
- `img_height=224`: All images are resized to **224×224 pixels** (standard for deep learning)
- `img_width=224`: Width of the image
- `self.model`: Stores the neural network
- `self.history`: Stores training history (loss & accuracy per epoch)

---

## 3️⃣ **Model Architecture: `create_model()` Method**

This is the **heart of the model**. Let me explain each layer:

### 📊 **Input Layer**


In [ ]:
layers.Input(shape=(224, 224, 3))

- Takes **224×224 RGB images** (3 color channels: Red, Green, Blue)

### 🔵 **Convolutional Block 1**


In [ ]:
layers.Conv2D(32, (3, 3), activation='relu', padding='same')
layers.BatchNormalization()
layers.MaxPooling2D((2, 2))
layers.Dropout(0.25)



**Conv2D (32 filters, 3×3 kernel)**:
- Scans the image with **32 different filters** (like pattern detectors)
- Each filter is a small **3×3 grid** that slides across the image
- `padding='same'` keeps image dimensions the same
- **ReLU activation**: Converts negative values to 0 (adds non-linearity)

**Why filters?** Each filter learns to detect different features:
- Filter 1 might detect **edges**
- Filter 2 might detect **textures**
- Filter 3 might detect **blurs**, etc.

**BatchNormalization**:
- Normalizes the output of each layer
- Makes training **faster and more stable**
- Helps prevent **internal covariate shift**

**MaxPooling2D (2×2)**:
- Reduces image size by taking the **maximum value** in each 2×2 window
- Reduces computation and helps detect features at different scales
- **224×224 → 112×112**

**Dropout(0.25)**:
- Randomly "turns off" 25% of neurons during training
- Prevents **overfitting** (memorizing the data instead of learning patterns)
- Acts as regularization

### 🟡 **Convolutional Block 2** (Same pattern, deeper features)


In [ ]:
layers.Conv2D(64, (3, 3), activation='relu', padding='same')
layers.BatchNormalization()
layers.MaxPooling2D((2, 2))
layers.Dropout(0.25)

- **64 filters** (more than Block 1 for more complex features)
- Image size: **112×112 → 56×56**
- Learns **second-level features** (combinations of first-level features)

### 🔴 **Convolutional Block 3** (Highest-level features)


In [ ]:
layers.Conv2D(128, (3, 3), activation='relu', padding='same')
layers.BatchNormalization()
layers.MaxPooling2D((2, 2))
layers.Dropout(0.25)

- **128 filters** (most complex patterns)
- Image size: **56×56 → 28×28**
- Learns **high-level medical patterns** (e.g., pneumonia patterns)

**Why 3 blocks?**
1. First block: **Low-level features** (edges, gradients)
2. Second block: **Mid-level features** (shapes, textures)
3. Third block: **High-level features** (medical patterns, anomalies)

### 🟣 **Classification Head** (Makes the final decision)


In [ ]:
layers.GlobalAveragePooling2D()

- Takes the 28×28×128 feature maps and **averages** them into a single value per filter
- Results in **128 values** (one per filter)



In [ ]:
layers.Dense(256, activation='relu')
layers.BatchNormalization()
layers.Dropout(0.5)

- **Dense layer**: Fully connected layer with 256 neurons
- Learns **complex combinations** of features
- **Dropout(0.5)**: Drop 50% (more aggressive, important for the final layer)



In [ ]:
layers.Dense(1, activation='sigmoid')

- **Final output**: 1 neuron with **sigmoid activation**
- Output range: **0 to 1**
- **0 = Normal**, **1 = Pneumonia**
- **Sigmoid**: Squashes values between 0-1 (probability-like)

---

## 4️⃣ **Model Compilation**



In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)



**Optimizer: Adam (learning_rate=0.001)**:
- **Adam** = Adaptive Moment Estimation
- Adjusts the learning rate automatically
- Learning rate 0.001 means small, controlled weight updates
- Too high = model diverges; Too low = slow training

**Loss: Binary Crossentropy**:
- Measures how far predictions are from ground truth
- For binary classification (2 classes only)
- Formula: `-(y*log(p) + (1-y)*log(1-p))`
- Model tries to **minimize this loss**

**Metrics: Accuracy**:
- Percentage of correct predictions
- **Accuracy = (TP + TN) / Total**

---

## 5️⃣ **Data Loading: `load_real_data()` Method**

### 📂 **Directory Structure Expected**


In [ ]:
data/
├── train/
│   ├── NORMAL/     (1341 images)
│   └── PNEUMONIA/  (~3500 images)
├── val/
│   ├── NORMAL/     (8 images)
│   └── PNEUMONIA/  (8 images)
└── test/
    ├── NORMAL/     (100 images)
    └── PNEUMONIA/  (100 images)



### 🖼️ **Image Processing**


In [ ]:
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (224, 224))

1. **Read** image using OpenCV
2. **Convert** from BGR (OpenCV format) to RGB
3. **Resize** to 224×224

### 🔢 **Normalization**


In [ ]:
X_train = np.array(train_normal_imgs + train_pneumonia_imgs, dtype=np.float32) / 255.0

- Divides pixel values by 255
- Converts range from **[0, 255] → [0, 1]**
- **Why?** Neural networks work better with normalized inputs

### 🔀 **Shuffling**


In [ ]:
shuffle_idx = np.random.permutation(len(X_train))
X_train = X_train[shuffle_idx]
y_train = y_train[shuffle_idx]

- Randomizes training order
- Prevents the model from learning the **sequence** instead of patterns

---

## 6️⃣ **Training: `train_model()` Method**



In [ ]:
self.history = self.model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=20,
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1
)



### 📊 **Key Parameters**
- **batch_size=32**: Process 32 images at a time
- **epochs=20**: Run through entire dataset 20 times
- **validation_data**: Check performance on unseen data each epoch
- **callbacks**: Control training behavior

### 🎯 **Callbacks**

**1. EarlyStopping**


In [ ]:
keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=4,
    restore_best_weights=True
)

- **Monitor**: Watch validation loss
- **Patience=4**: Stop if no improvement for 4 consecutive epochs
- Prevents **overfitting** and **wastes training time**

**2. ReduceLROnPlateau**


In [ ]:
keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=2,
    min_lr=0.00001
)

- **If** validation loss plateaus for 2 epochs
- **Then** multiply learning rate by 0.5 (reduce to half)
- Takes smaller steps when stuck
- Minimum learning rate: 0.00001

---

## 7️⃣ **Evaluation: `evaluate_model()` Method**

### 🎲 **Getting Predictions**


In [ ]:
predictions = self.model.predict(X_test, verbose=0)
y_pred = (predictions > 0.5).astype(int).flatten()

- Model outputs values 0-1 (probability)
- **> 0.5 = Pneumonia (1)**, **≤ 0.5 = Normal (0)**

### 📈 **Performance Metrics**



In [ ]:
Accuracy = (TP + TN) / Total
Precision = TP / (TP + FP)
Recall = TP / (TP + FN)
F1 Score = 2 * (Precision * Recall) / (Precision + Recall)



**Example Scenario (100 test images: 50 Normal, 50 Pneumonia)**
- **TP=45**: Correctly predicted Pneumonia
- **FP=5**: Predicted Pneumonia, actually Normal
- **FN=5**: Predicted Normal, actually Pneumonia
- **TN=45**: Correctly predicted Normal

| Metric | Value | Interpretation |
|--------|-------|-----------------|
| Accuracy | 90% | Gets 90% of cases right |
| Precision | 90% | When predicting Pneumonia, 90% correct |
| Recall | 90% | Catches 90% of actual Pneumonia cases |
| F1 Score | 90% | Balanced measure of precision & recall |

---

## 8️⃣ **Real-World Performance** (From Your Training)



In [ ]:
Training Accuracy:   96%  ← Very high (saw many examples)
Validation Accuracy: 87.5% ← Lower (unseen during training)
Test Accuracy:       74%   ← Actual real-world performance



**What this tells us:**
- ✅ Model learned well (training high)
- ⚠️ Some overfitting (validation lower than training)
- ❓ Lower test accuracy suggests dataset differences

---

## 9️⃣ **Why This Architecture Works**

| Component | Purpose |
|-----------|---------|
| **Conv2D blocks** | Extract features at multiple scales |
| **BatchNormalization** | Stabilize training & faster convergence |
| **MaxPooling** | Reduce spatial dimensions, focus on important features |
| **Dropout** | Prevent overfitting |
| **GlobalAveragePooling** | Combine all feature maps smoothly |
| **Dense layers** | Learn complex relationships |
| **Sigmoid output** | Convert to probability (0-1) |

---

## 🔟 **Complete Training Pipeline**



In [ ]:
1. Initialize classifier
   ↓
2. Create CNN model (117 layers total)
   ↓
3. Load 1341 Normal + 3500 Pneumonia images
   ↓
4. Train on 4841 images for 20 epochs max
   ↓
5. Validate on 16 images each epoch
   ↓
6. Stop early if not improving
   ↓
7. Evaluate on 200 test images
   ↓
8. Save model & training plots



---

## 📝 **Key Takeaways**

✅ **CNN** learns hierarchical features automatically  
✅ **Dropout** prevents memorization (overfitting)  
✅ **Batch Normalization** stabilizes training  
✅ **Early Stopping** prevents wasting time  
✅ **Binary classification** with sigmoid output  
✅ **Real medical data** beats synthetic data  

Would you like me to explain any specific part in more detail?